# Day - Airflow

[Airflow](https://airflow.apache.org/) is a platform to programmatically author, schedule and monitor workflows. This is used beyond Machine Learning but it is very useful tool when you put Machine Learning in production.

In this exercise, you will learn how to setup Airflow with GCP, create your first [DAG](https://en.wikipedia.org/wiki/Directed_acyclic_graph) and finally setup a pipeline for the Taxi Fare Prediction Challenge 

<img src="https://ncrocfer.github.io/images/airflow-logo.png" width="400" />

## Summary

1. [Airflow setup with GCP](#part1)
2. [Your first DAG](#part2)
3. [DAG with dependencies](#part3)
4. [Taxi Fare Prediction Pipeline](#part4)
5. [Data Enriching](#part5)
5. [Real Use Case](#part6)

## 1. Airflow Setup with GCP<a id="part1" />

Google Cloud has a tool called `Composer` that manages Airflow for you! **Let's use that!**

#### Exercise
- Create a [Gloud Composer]((https://console.cloud.google.com/composer) account
- Create a new environment
- Finally, go to Airflow Web Server interface to check it out.

_Note : Airflow uses Compute Engine resources (3 by default), it is expensive if you let it run for a week, so don't forget to delete your Airflow session at the end of the day._

## 2. Your first DAG<a id="part2" />

#### Exercise

- Write your `first_dag.py`
- Fill in the `test()` function
- Instantiate a `DAG` that will run every minute
- Add an operator to the dag that calls the `test` function
- Write a Makefile to updload this DAG to GCP

In [14]:
## first_dag.py

import datetime
from airflow import DAG
from airflow.operators.python_operator import PythonOperator

def test():
    """test method"""
    pass
    

## create DAG here
dag = DAG()

## test operator
test_opr = PythonOperator()

## 3. DAG with dependencies<a id="part3" />

## 4. Taxi Fare Prediction Pipeline<a id="part4" />

## 5. Data Enriching<a id="part5" />

## 6. Real Use Case<a id="part6" />